In [4]:
import ctd
import gsw
from ctd.read import _basename
import requests
import os
import re
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

# Make the Plots pretty
import seaborn as sns

In [1]:
def derive_cnv(cast):
    """Compute SP, SA, CT, z, and GP from a cnv pre-processed cast."""
    p = cast.index.values.astype(float)
    lon = cast._metadata.get("lon")
    lat = cast._metadata.get("lat")

    SP = gsw.SP_from_C(cast["c0S/m"] * 10., cast["t090C"], p)
    cast["SA"] = gsw.SA_from_SP(SP, p, lon, lat)
    cast["CT"] = gsw.CT_from_t(cast["SA"], cast["t090C"], p)
    cast["z"] = -gsw.z_from_p(p, lat)
    cast["sigma0_CT"] = gsw.sigma0(cast["SA"], cast["CT"])
    return cast


def proc_ctd(fname):
    # read and fill metadata
    cast = ctd.from_cnv(fname).split()[0]

    name = _basename(str(fname))[1]
    lat = cast["latitude"].mean()
    lon = cast["longitude"].mean()

    # 00-split, clean "bad pump" data, and apply flag
    cast = cast[cast["pumps"]]
    cast = cast[~cast["flag"]]  # True for bad values.

    # 03-loop edit
    cast["dz/dtM"] = cast["dz/dtM"].movingaverage(window_size=48)
    cast = cast[cast["dz/dtM"] >= 0.25]  # Threshold velocity.

    # from now on we'll only need in site temperature and conductivity.
    cols = [ "t090C", "c0S/m"]
    cast = cast[cols]

    cast = cast.remove_above_water()\
               .lp_filter()\
               .press_check()\
               .dropna()\
               .despike(n1=2, n2=20, block=100)\
    
    cast._metadata = {"name": name, "lat": lat, "lon": lon}

    # 08-Derive.
    cast = derive_cnv(cast)
    
    cast = cast.set_index("z")
    cast = cast.bindata(delta=1, method="interpolate")
    cast._metadata = {"name": name, "lat": lat, "lon": lon}
    return cast

In [7]:
data_url = {}
data_url['inshore'] = 'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/sbeaulieu@whoi.edu/20200806T132326640Z-CP03ISPM-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered/deployment0003_CP03ISPM-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered_20191006T150003-20191031T212239.977728.nc#fillmismatch'
data_url['central_inshore'] = 'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/sbeaulieu@whoi.edu/20200806T132900316Z-CP02PMCI-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered/deployment0013_CP02PMCI-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered_20191007T210003-20191031T212442.986087.nc#fillmismatch'
data_url['central_offshore'] = 'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/sbeaulieu@whoi.edu/20200806T133142674Z-CP02PMCO-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered/deployment0013_CP02PMCO-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered_20191008T140003-20191031T212529.983845.nc#fillmismatch'
data_url['offshore'] = 'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/sbeaulieu@whoi.edu/20200806T133343088Z-CP04OSPM-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered/deployment0012_CP04OSPM-WFP01-03-CTDPFK000-recovered_wfp-ctdpf_ckl_wfp_instrument_recovered_20191013T160003-20191031T211622.990750.nc#fillmismatch'

# Load the data file using xarray
def load2xarray(location):
    """
    Load data at given location and reduce to variables of interest.
    """
    ds = xr.open_dataset(data_url[location])
    ds = ds.swap_dims({'obs': 'time'}) #Swap dimensions
    print('Dataset '+ location +' has %d points' % ds.time.size)
    ds = ds[['ctdpf_ckl_seawater_pressure','ctdpf_ckl_seawater_temperature','practical_salinity']]
    return ds

ds={}
for loc in list(data_url.keys()):
    ds[loc] = load2xarray(loc)

Dataset inshore has 158237 points
Dataset central_inshore has 210513 points
Dataset central_offshore has 236989 points
Dataset offshore has 199587 points


In [8]:
ds["inshore"]

<xarray.Dataset>
Dimensions:                         (time: 158237)
Coordinates:
  * time                            (time) datetime64[ns] 2019-10-06T15:00:03...
    lon                             (time) float64 ...
    obs                             (time) int32 0 1 2 ... 158234 158235 158236
    pressure                        (time) float64 ...
    lat                             (time) float64 ...
Data variables:
    ctdpf_ckl_seawater_pressure     (time) float64 ...
    ctdpf_ckl_seawater_temperature  (time) float64 ...
    practical_salinity              (time) float64 ...
Attributes:
    _NCProperties:                   version=1|netcdflibversion=4.6.1|hdf5lib...
    node:                            WFP01
    comment:                         
    publisher_email:                 
    sourceUrl:                       http://oceanobservatories.org/
    collection_method:               recovered_wfp
    stream:                          ctdpf_ckl_wfp_instrument_recovered
    featureType:                     point
    creator_email:                   
    publisher_name:                  Ocean Observatories Initiative
    date_modified:                   2020-08-06T13:23:41.238357
    keywords:                        
    cdm_data_type:                   Point
    references:                      More information can be found at http://...
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    date_created:                    2020-08-06T13:23:41.238351
    id:                              CP03ISPM-WFP01-03-CTDPFK000-recovered_wf...
    requestUUID:                     607b59bd-cfc8-4659-868c-a894a197fb41
    contributor_role:                
    summary:                         Dataset Generated by Stream Engine from ...
    keywords_vocabulary:             
    institution:                     Ocean Observatories Initiative
    naming_authority:                org.oceanobservatories
    feature_Type:                    point
    infoUrl:                         http://oceanobservatories.org/
    license:                         
    contributor_name:                
    uuid:                            607b59bd-cfc8-4659-868c-a894a197fb41
    creator_name:                    Ocean Observatories Initiative
    title:                           Data produced by Stream Engine version 1...
    sensor:                          03-CTDPFK000
    standard_name_vocabulary:        NetCDF Climate and Forecast (CF) Metadat...
    acknowledgement:                 
    Conventions:                     CF-1.6
    project:                         Ocean Observatories Initiative
    source:                          CP03ISPM-WFP01-03-CTDPFK000-recovered_wf...
    publisher_url:                   http://oceanobservatories.org/
    creator_url:                     http://oceanobservatories.org/
    nodc_template_version:           NODC_NetCDF_TimeSeries_Orthogonal_Templa...
    subsite:                         CP03ISPM
    processing_level:                L2
    history:                         2020-08-06T13:23:41.238281 generated fro...
    time_coverage_start:             2019-10-06T15:00:03
    time_coverage_end:               2019-10-31T21:22:39.977728
    time_coverage_resolution:        P13.80S
    geospatial_lat_min:              40.364817
    geospatial_lat_max:              40.364817
    geospatial_lat_units:            degrees_north
    geospatial_lat_resolution:       0.1
    geospatial_lon_min:              -70.888668
    geospatial_lon_max:              -70.888668
    geospatial_lon_units:            degrees_east
    geospatial_lon_resolution:       0.1
    geospatial_vertical_units:       meters
    geospatial_vertical_resolution:  0.1
    geospatial_vertical_positive:    down
    DODS.strlen:                     18
    DODS.dimName:                    string18
    DODS_EXTRA.Unlimited_Dimension:  obs

In [ ]:
from pathlib import Path


path = Path("..", "tests", "data", "CTD")


def alphanum_key(s):
    import re
    s = str(s)
    key = re.split(r"(\d+)", s)
    key[1::2] = map(int, key[1::2])
    return key

fnames = sorted(
    path.glob("*c.cnv.gz"),
    key=alphanum_key
)

ctd_casts = [proc_ctd(fname) for fname in fnames]


In [ ]:
lons, lats = [], []

for cast in ctd_casts:
    lons.append(cast._metadata.get("lon"))
    lats.append(cast._metadata.get("lat"))

dx = dy = 2
bbox = min(lons)-dx, max(lons)+dx, min(lats)-dy, max(lats)+dy

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs


# https://ocefpaf.github.io/python4oceanographers/blog/2013/07/29/python-ctd/
projection = ccrs.PlateCarree()
fig, ax = plt.subplots(subplot_kw={"projection": projection})
ax.set_extent(bbox)
ax.coastlines()
ax.plot(lons, lats, "ro");

In [ ]:
from ctd.extras import *

from gsw import distance
import pandas as pd


def plot_section(ctd_casts, var, reverse=False, **kw):
    names = [df._metadata.get("name") for df in ctd_casts]
    lons, lats, data = [], [], []
    for cast in ctd_casts:
        lons.append(cast._metadata.get("lon"))
        lats.append(cast._metadata.get("lat"))
        data.append(cast[var])


    sec = pd.concat(data, axis=1)
    sec.columns = names

    z = sec.index.values
    h = get_maxdepth(sec)

    lons, lats = map(np.atleast_2d, (lons, lats))
    x = np.append(0, np.cumsum(distance(lons, lats)[0] / 1e3))
    z = sec.index.values.astype(float)
    data = ma.masked_invalid(sec)

    if reverse:
        lons = lons[::-1]
        lats = lats[::-1]
        data = data.T[::-1].T
        h = h[::-1]

    # Contour key words.
    extend = kw.pop("extend", "both")
    fontsize = kw.pop("fontsize", 12)
    labelsize = kw.pop("labelsize", 11)
    cmap = kw.pop("cmap", None)
    levels = kw.pop(
        "levels", np.arange(np.floor(np.nanmin(data)), np.ceil(np.nanmax(data)) + 0.5, 0.5),
    )

    # Colorbar key words.
    pad = kw.pop("pad", 0.04)
    aspect = kw.pop("aspect", 40)
    shrink = kw.pop("shrink", 0.9)
    fraction = kw.pop("fraction", 0.05)

    # Topography mask key words.
    dx = kw.pop("dx", 1.0)
    kind = kw.pop("kind", "linear")
    linewidth = kw.pop("linewidth", 1.5)

    # Station symbols key words.
    station_marker = kw.pop("station_marker", None)
    color = kw.pop("color", "k")
    offset = kw.pop("offset", -5)
    alpha = kw.pop("alpha", 0.5)

    # Figure.
    figsize = kw.pop("figsize", (12, 6))
    fig, ax = plt.subplots(figsize=figsize)
    xm, hm = gen_topomask(h, lons, lats, dx=dx, kind=kind)
    ax.plot(xm, hm, color="black", linewidth=linewidth, zorder=3)
    ax.fill_between(xm, hm, y2=hm.max(), color="0.9", zorder=3)

    if station_marker:
        ax.plot(
            x,
            [offset] * len(h),
            color=color,
            marker=station_marker,
            alpha=alpha,
            zorder=5,
        )
    ax.set_xlabel("Cross-shore distance [km]", fontsize=fontsize)
    ax.set_ylabel("Depth [m]", fontsize=fontsize)
    ax.set_ylim(offset, hm.max())
    ax.invert_yaxis()

    ax.xaxis.set_ticks_position("top")
    ax.xaxis.set_label_position("top")
    ax.yaxis.set_ticks_position("left")
    ax.yaxis.set_label_position("left")
    ax.xaxis.set_tick_params(tickdir="out", labelsize=labelsize, pad=1)
    ax.yaxis.set_tick_params(tickdir="out", labelsize=labelsize, pad=1)

    cs = ax.contourf(
        x, z, data, cmap=cmap, levels=levels, alpha=1.0, extend=extend, zorder=2,
    )

    cb = fig.colorbar(
        mappable=cs,
        ax=ax,
        orientation="vertical",
        aspect=aspect,
        shrink=shrink,
        fraction=fraction,
        pad=pad,
    )
    return fig, ax, cb


fig, ax, cb = plot_section(ctd_casts, var="CT", reverse=True, filled=True)